# Bigquery Version

In [ ]:
# prompt: how to attach a screenshot in the cell?

# from IPython.display import Image
# Image(url="https://raw.githubusercontent.com/AlwaysSany/bigquery-hackathon/refs/heads/main/public_dataset_from_bigquery.png?token=GHSAT0AAAAAADHZYTMY7I5W2OC4FOURBGEU2GK6QKQ")

### Understand Product Structure Stats

In [1]:
%%bigquery
-- This query counts the total number of products, unique categories, brands, and departments
-- in the `products` table to understand the product structure.
SELECT
  COUNT(*) as total_products,
  COUNT(DISTINCT category) as categories,
  COUNT(DISTINCT brand) as brands,
  COUNT(DISTINCT department) as departments
FROM `bigquery-public-data.thelook_ecommerce.products`;

Query is running:   0%|          |

Downloading:   0%|          |

,total_products,categories,brands,departments
0,29120,26,2756,2


### Sample Product Structure

In [2]:
%%bigquery
-- This query selects a sample of 5 rows from the `products` table
-- to provide a glimpse into the product structure.
SELECT
  id,
  name,
  category,
  brand,
  department,
  retail_price,
  cost
FROM `bigquery-public-data.thelook_ecommerce.products`
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,category,brand,department,retail_price,cost
0,13842,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,Accessories,MG,Women,6.25,2.51875
1,13928,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,Accessories,MG,Women,5.95,2.33835
2,14115,Enzyme Regular Solid Army Caps-Black W35S45D,Accessories,MG,Women,10.99,4.87956
3,14157,Enzyme Regular Solid Army Caps-Olive W35S45D (...,Accessories,MG,Women,10.99,4.64877
4,14273,Washed Canvas Ivy Cap - Black W11S64C,Accessories,MG,Women,15.99,6.50793


### Dataset Overview

In [3]:
%%bigquery
-- This query provides a general overview of the dataset,
-- including the total number of products, unique categories, brands,
-- departments, and price statistics.
SELECT
  '📊 TheLook E-commerce Dataset Overview' as analysis,
  COUNT(*) as total_products,
  COUNT(DISTINCT category) as categories,
  COUNT(DISTINCT brand) as brands,
  COUNT(DISTINCT department) as departments,
  AVG(retail_price) as avg_price,
  MIN(retail_price) as min_price,
  MAX(retail_price) as max_price
FROM `bigquery-public-data.thelook_ecommerce.products`
WHERE name IS NOT NULL;

Query is running:   0%|          |

Downloading:   0%|          |

,analysis,total_products,categories,brands,departments,avg_price,min_price,max_price
0,📊 TheLook E-commerce Dataset Overview,29118,26,2756,2,59.217913,0.02,999.0


### Create Product Features Table

In [ ]:
%%bigquery
-- Creates a table `product_features` with product information and a semantic description.
-- The semantic description concatenates product details and a price range category.
CREATE OR REPLACE TABLE `bigquery-hackathon-471715.thelook_ecommerce.product_features` AS
SELECT
  p.id,
  p.name,
  p.category,
  p.brand,
  p.department,
  p.retail_price,
  p.cost,
  CONCAT(
    'Product: ', p.name, '. ',
    'Category: ', p.category, '. ',
    'Brand: ', p.brand, '. ',
    'Department: ', p.department, '. ',
    'Price range: ',
    CASE
      WHEN p.retail_price < 20 THEN 'budget-friendly'
      WHEN p.retail_price < 50 THEN 'mid-range'
      WHEN p.retail_price < 100 THEN 'premium'
      ELSE 'luxury'
    END
  ) AS semantic_description
FROM `bigquery-public-data.thelook_ecommerce.products` p;

### Create Text Embedding Model

In [ ]:
%%bigquery
-- Creates a remote model for text embedding using the 'text-embedding-004' endpoint.
CREATE MODEL `bigquery-hackathon-471715.thelook_ecommerce.thelook_model_text_embedding_004`
REMOTE WITH CONNECTION DEFAULT
OPTIONS(
  ENDPOINT = 'text-embedding-004'
);

### Bigquery: Generate Product Embeddings

In [ ]:
%%bigquery
-- Generates text embeddings for the semantic descriptions in the `product_features` table
-- using the created embedding model and stores them in `product_embeddings`.
CREATE OR REPLACE TABLE `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings` AS
SELECT
  *,
  ml_generate_embedding_result AS embedding
FROM
  ML.GENERATE_EMBEDDING(
    MODEL `bigquery-hackathon-471715.thelook_ecommerce.thelook_model_text_embedding_004`,
    (
      SELECT
        id,
        name,
        category,
        brand,
        department,
        retail_price,
        semantic_description AS content
      FROM `bigquery-hackathon-471715.thelook_ecommerce.product_features`
    )
  );

### Bigquery: Clean Product Embeddings

In [ ]:
%%bigquery
--
CREATE OR REPLACE TABLE `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` AS
SELECT *
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings`
WHERE embedding IS NOT NULL
  AND ARRAY_LENGTH(embedding) = 768;

### Bigquery: Create Vector Index

In [ ]:
%%bigquery
CREATE VECTOR INDEX product_similarity_index
ON `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`(embedding)
OPTIONS(
  index_type='IVF',
  distance_type='COSINE',
  ivf_options='{"num_lists": 1000}'
);

### Target product structure

In [8]:
%%bigquery
-- This query shows the structure of the `products` table.
SELECT * FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` WHERE id = 21018;

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,id,name,category,brand,department,retail_price,content,embedding
0,"[0.032505184412002563, -0.024057593196630478, ...","{""token_count"":36,""truncated"":false}",,21018,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,Men,49.990002,Product: Wrangler Men's Western Traditional Bo...,"[0.032505184412002563, -0.024057593196630478, ..."


### Normal Search

In [16]:
%%bigquery
WITH target_product AS (
  SELECT category, brand
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
)
SELECT
  base.id,
  base.name,
  base.category,
  base.brand,
  base.retail_price
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` base
JOIN target_product target
  ON base.category = target.category
  AND base.brand = target.brand
WHERE base.id != 21018
ORDER BY base.retail_price ASC
LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,id,name,category,brand,retail_price
0,20674,Genuine Wrangler Men's Relaxed Fit Jean,Jeans,Wrangler,25.00
1,20686,Genuine Wrangler Men's Loose Fit Jean,Jeans,Wrangler,25.00
2,4341,Wrangler Blues Women's Relaxed Jean,Jeans,Wrangler,25.90
3,4407,Wrangler Women's Cowgirl Cut Mid-Rise Ultimate...,Jeans,Wrangler,26.65
4,21004,Genuine Wrangler Men's Relaxed Fit Jean,Jeans,Wrangler,29.99


### Test 1: Basic Similar Product(semantic)

In [17]:
%%bigquery
WITH target_product AS (
  SELECT embedding, name as target_name, category, brand, retail_price
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
)
SELECT
  base.id,
  base.name,
  target.target_name,
  base.category,
  base.brand,
  base.retail_price,
  ROUND((1 - COSINE_DISTANCE(target.embedding, base.embedding)), 3) AS similarity_score
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` base
CROSS JOIN target_product target
WHERE base.id != 21018
ORDER BY similarity_score DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,target_name,category,brand,retail_price,similarity_score
0,21284,Wrangler Men's Western Boot Cut Slim Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,35.950001,0.990
1,20850,Wrangler Men's Western Slim Boot Cut Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.990
2,20921,Wrangler Men's Cowboy Cut Western Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,40.720001,0.974
3,21255,Wrangler Men's Western Boot Cut Jean Regular,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,37.000000,0.972
4,20585,Wrangler Men's Cowboy Cut Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.967


### Test 2: Smart Multi-Factor Recommendations(semantic)

In [35]:
%%bigquery
WITH target_product AS (
  SELECT
    embedding, retail_price, category, brand, name as target_name
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
),
smart_recommendations AS (
  SELECT
    p.id,
    p.name,
    t.target_name,
    p.category,
    p.brand,
    p.retail_price,
    ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) * 0.6 AS semantic_score,
    ROUND((1 - ABS(p.retail_price - t.retail_price) / GREATEST(p.retail_price, t.retail_price)), 3) * 0.25 AS price_score,
    CASE WHEN p.category = t.category THEN 0.1 ELSE 0 END AS category_bonus,
    CASE WHEN p.brand = t.brand THEN 0.05 ELSE 0 END AS brand_bonus
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
  CROSS JOIN target_product t
  WHERE p.id != 21018
)
SELECT *,
  ROUND((semantic_score + price_score + category_bonus + brand_bonus), 3) AS total_score,
  CASE
    WHEN category_bonus > 0 AND brand_bonus > 0 THEN 'Perfect Match'
    WHEN category_bonus > 0 THEN 'Same Category'
    WHEN brand_bonus > 0 THEN 'Same Brand'
    WHEN semantic_score > 0.4 THEN 'Semantically Similar'
    ELSE 'Alternative'
  END AS match_type
FROM smart_recommendations
ORDER BY total_score DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,target_name,category,brand,retail_price,semantic_score,price_score,category_bonus,brand_bonus,total_score,match_type
0,20850,Wrangler Men's Western Slim Boot Cut Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.5940,0.25,0.1,0.05,0.994,Perfect Match
1,20585,Wrangler Men's Cowboy Cut Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.5802,0.25,0.1,0.05,0.980,Perfect Match
2,20632,Wrangler Men's Cowboy Cut Original Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.5676,0.25,0.1,0.05,0.968,Perfect Match
3,20968,Wrangler Men's Original Cowboy Cut Relaxed Fit...,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.5640,0.25,0.1,0.05,0.964,Perfect Match
4,20971,Wrangler Men's Premium Performance Cowboy Cut ...,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,0.5586,0.25,0.1,0.05,0.959,Perfect Match


### Test 3: Cross-Category Discovery

In [34]:
%%bigquery
WITH target AS (
  SELECT embedding,
  category as target_category,
  name as target_name,
  brand,
  retail_price
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
)
SELECT
  p.id, p.name, t.target_name,
  p.category, t.target_category,
  p.brand, p.retail_price,
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) AS similarity,
  CASE WHEN p.category = t.target_category THEN 'Same' ELSE 'Different' END AS category_match
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
CROSS JOIN target t
WHERE p.id != 21018
  AND p.category != t.target_category
ORDER BY similarity DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,target_name,category,target_category,brand,retail_price,similarity,category_match
0,22035,Men's Wrangler George Straight Cowboy Cut Casu...,Wrangler Men's Western Traditional Boot Cut Sl...,Pants,Jeans,Wrangler,34.990002,0.871,Different
1,21762,Wrangler Men's Ranger Pant,Wrangler Men's Western Traditional Boot Cut Sl...,Pants,Jeans,Wrangler,49.880001,0.862,Different
2,21993,Wrangler Men's Ranger Pant,Wrangler Men's Western Traditional Boot Cut Sl...,Pants,Jeans,Wrangler,49.880001,0.862,Different
3,23363,Wrangler Men's Ranger Short,Wrangler Men's Western Traditional Boot Cut Sl...,Shorts,Jeans,Wrangler,37.000000,0.844,Different
4,22888,Wrangler Men's Rugged Wear Relaxed Fit Short,Wrangler Men's Western Traditional Boot Cut Sl...,Shorts,Jeans,Wrangler,25.000000,0.842,Different


### Test 4: Price-Conscious Recommendations(semantic)

In [33]:
%%bigquery
WITH target AS (
  SELECT embedding,
  retail_price,
  name,
  category,
  brand
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
)
SELECT
  p.id, p.name, t.name as original_product,
  p.category, p.brand,
  p.retail_price, t.retail_price as original_price,
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) AS similarity,
  ROUND(((p.retail_price - t.retail_price) / t.retail_price * 100), 1) AS price_change_pct,
  CASE
    WHEN p.retail_price < t.retail_price THEN 'Cheaper'
    WHEN p.retail_price = t.retail_price THEN 'Same Price'
    ELSE 'Premium'
  END AS price_category
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
CROSS JOIN target t
WHERE p.id != 21018
  AND ABS(p.retail_price - t.retail_price) / t.retail_price <= 0.2
ORDER BY similarity DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,original_product,category,brand,retail_price,original_price,similarity,price_change_pct,price_category
0,20850,Wrangler Men's Western Slim Boot Cut Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,49.990002,0.990,0.0,Same Price
1,20921,Wrangler Men's Cowboy Cut Western Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,40.720001,49.990002,0.974,-18.5,Cheaper
2,20585,Wrangler Men's Cowboy Cut Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,49.990002,49.990002,0.967,0.0,Same Price
3,21181,Wrangler Men's Cowboy Cut Original Fit Western...,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,44.950001,49.990002,0.952,-10.1,Cheaper
4,20978,Wrangler Men's Retro Boot Cut Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,45.419998,49.990002,0.949,-9.1,Cheaper


### Test 5: Trend-Aware Recommendations(semantic)

##### **ordert_items** table structure

In [26]:
%%bigquery
SELECT *
  FROM `bigquery-public-data.thelook_ecommerce.order_items`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,139472,95848,76933,14235,376492,Cancelled,2022-08-13 05:35:33+00:00,NaT,NaT,NaT,0.02
1,51114,35221,28334,14235,137910,Returned,2024-09-09 06:39:52+00:00,2024-09-09 04:26:00+00:00,2024-09-12 05:58:00+00:00,2024-09-12 18:03:00+00:00,0.02
2,92702,63771,51221,14159,250292,Cancelled,2025-04-18 04:24:30+00:00,NaT,NaT,NaT,0.49
3,27019,18695,15115,14159,73103,Shipped,2024-12-14 02:34:28+00:00,2024-12-10 22:08:00+00:00,NaT,NaT,0.49
4,86405,59446,47783,14159,233340,Shipped,2025-03-18 14:22:11+00:00,2025-03-18 13:46:00+00:00,NaT,NaT,0.49


In [32]:
%%bigquery
WITH target AS (
  SELECT id,
  embedding,
  name,
  category,
  brand,
  retail_price
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
),
popularity AS (
  SELECT product_id, COUNT(*) AS order_count
  FROM `bigquery-public-data.thelook_ecommerce.order_items`
  GROUP BY product_id
)
SELECT
  p.id,
  p.name,
  t.name as original_product,
  p.category,
  p.brand,
  p.retail_price,
  COALESCE(pop.order_count, 0) AS popularity,
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) AS similarity,
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) * 0.7 +
  LOG10(COALESCE(pop.order_count, 1)) * 0.3 AS weighted_score
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
LEFT JOIN popularity pop ON p.id = pop.product_id
CROSS JOIN target t
WHERE p.id != t.id
ORDER BY weighted_score DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,original_product,category,brand,retail_price,popularity,similarity,weighted_score
0,21255,Wrangler Men's Western Boot Cut Jean Regular,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,37.000000,9,0.972,0.966673
1,21405,Wrangler Men's Premium Performance Cowboy Cut ...,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,47.990002,11,0.931,0.964118
2,21151,Wrangler Men's Genuine Relaxed Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,40.000000,11,0.925,0.959918
3,20732,Wrangler Men's Cowboy Cut Original Fit Western...,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,38.980000,9,0.952,0.952673
4,20659,Wrangler Men's Rugged Wear Classic Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Jeans,Wrangler,29.990000,10,0.932,0.952400


### Test 6: Department-Level Exploration(semantic)

In [31]:
%%bigquery
WITH target AS (
  SELECT embedding, department as target_department, name as target_name
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
)
SELECT
  p.id, p.name, t.target_name,
  p.department, t.target_department,
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) AS similarity
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
CROSS JOIN target t
WHERE p.id != 21018
  AND p.department != t.target_department
ORDER BY similarity DESC
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,target_name,department,target_department,similarity
0,4993,Wrangler Womens Western Low Rise Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Women,Men,0.890
1,4412,Wrangler Women's As Real as Wrangler Classic F...,Wrangler Men's Western Traditional Boot Cut Sl...,Women,Men,0.880
2,4567,Wrangler Women's Cowgirl Cut Low Rise Ultimate...,Wrangler Men's Western Traditional Boot Cut Sl...,Women,Men,0.867
3,4407,Wrangler Women's Cowgirl Cut Mid-Rise Ultimate...,Wrangler Men's Western Traditional Boot Cut Sl...,Women,Men,0.863
4,4341,Wrangler Blues Women's Relaxed Jean,Wrangler Men's Western Traditional Boot Cut Sl...,Women,Men,0.850


## Test 7: Inventory-Level in-Stock Substitutes (semantic)

##### **inventory** table structure

In [29]:
%%bigquery
SELECT * from `bigquery-public-data.thelook_ecommerce.inventory_items` LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,178984,13844,2025-06-04 06:52:03+00:00,2025-07-28 04:06:03+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,178985,13844,2021-06-27 11:17:00+00:00,NaT,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,240803,13844,2024-05-05 00:41:58+00:00,2024-06-27 13:07:58+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,240804,13844,2023-09-23 14:45:00+00:00,NaT,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,253194,13844,2024-12-27 17:07:14+00:00,2025-02-23 16:47:14+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


In [38]:
%%bigquery
-- This query finds similar products that are currently in stock,
-- using the inventory_items table to check stock status.
WITH target AS (
  SELECT id, embedding, name
  FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean`
  WHERE id = 21018
),
inventory_status AS (
  -- Calculates the number of actively stocked items for each product.
  SELECT
    product_id,
    COUNTIF(sold_at IS NULL OR sold_at > CURRENT_TIMESTAMP()) AS active_stock
  FROM `bigquery-public-data.thelook_ecommerce.inventory_items`
  GROUP BY product_id
)
SELECT
  p.id, p.name, t.name AS original_product,
  -- Calculates the cosine similarity score between the target product and others.
  ROUND((1 - COSINE_DISTANCE(t.embedding, p.embedding)), 3) AS similarity,
  -- Determines the stock status based on active stock count.
  CASE
    WHEN COALESCE(inv.active_stock, 0) > 0 THEN '✅ In Stock'
    ELSE '❌ Out of Stock'
  END AS stock_status
FROM `bigquery-hackathon-471715.thelook_ecommerce.product_embeddings_clean` p
CROSS JOIN target t
LEFT JOIN inventory_status inv ON p.id = inv.product_id
WHERE p.id != t.id -- Excludes the target product from the results.
  AND COALESCE(inv.active_stock, 0) > 0 -- Filters for products that are in stock.
ORDER BY similarity DESC -- Orders the results by similarity score in descending order.
LIMIT 5; -- Limits the output to the top 5 most similar in-stock products.

Query is running:   0%|          |

Downloading:   0%|          |

,id,name,original_product,similarity,stock_status
0,21284,Wrangler Men's Western Boot Cut Slim Jean,Wrangler Men's Western Traditional Boot Cut Sl...,0.990,✅ In Stock
1,20850,Wrangler Men's Western Slim Boot Cut Jean,Wrangler Men's Western Traditional Boot Cut Sl...,0.990,✅ In Stock
2,20921,Wrangler Men's Cowboy Cut Western Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,0.974,✅ In Stock
3,21255,Wrangler Men's Western Boot Cut Jean Regular,Wrangler Men's Western Traditional Boot Cut Sl...,0.972,✅ In Stock
4,20585,Wrangler Men's Cowboy Cut Slim Fit Jean,Wrangler Men's Western Traditional Boot Cut Sl...,0.967,✅ In Stock
